In [ ]:
import numpy as np
import pandas as pd
import cvxopt
from cvxopt import matrix, solvers
import math
import matplotlib.pyplot as plt
from google.colab import output
solvers.options['show_progress'] = False

In [ ]:
def mean_variance_model_cvxopt(asset_prices, R, S):
  if (S==1):
    ww = np.zeros(len(asset_prices[0]))
    ww[0] = 1
    return 1000000, ww
  temparr = asset_prices[:,:S]
  return_rate = (temparr[:-1] - temparr[1:])/temparr[1:]
  exp_return = np.mean(return_rate,axis=0)
  temp_cov = np.cov(np.transpose(return_rate))
  num_companies = S

  temp_cov += np.eye(num_companies) * 1e-6
  e = np.ones(num_companies)

  P = matrix(2*temp_cov)
  q = matrix(np.zeros(num_companies))

  A = matrix(np.array([e, exp_return]))
  b = matrix(np.array([1,R]))

  G = matrix(-np.eye(num_companies))
  h = matrix(np.zeros(num_companies))
  sol = solvers.qp(P, q, G=G, h=h, A=A, b=b)

  wmw = np.array(sol['x'])
  wmw = wmw/sum(wmw)

  mkw_risk = math.sqrt(wmw.T@temp_cov@wmw)

  for _ in range(len(asset_prices[0])-S):
    wmw = np.append(wmw,0)

  return mkw_risk, wmw

In [ ]:
def lowerLevelSolutionArray(ogarr, candidate_returns, candidate_number_of_assets):

  n = len(candidate_returns)
  minRisks = []
  minRiskWeights = []

  for ind in range(n):
    solInd, ww = mean_variance_model_cvxopt(ogarr,candidate_returns[ind],candidate_number_of_assets[ind])
    minRisks.append(solInd)
    minRiskWeights.append(ww)

  return np.array(minRisks), minRiskWeights

In [ ]:
def calculate_returns(prices):
  """Calculate daily returns from prices."""
  return (prices[:-1] - prices[1:]) / prices[1:]

def calculate_portfolio_returns(returns, weights):
  """Calculate portfolio returns given asset returns and weights."""
  return np.dot(returns, weights)

def calculate_moments(returns):
  """Calculate the first four moments of returns."""
  n = len(returns)
  mean = np.sum(returns) / n
  variance = np.sum((returns - mean) ** 2) / n
  skewness = np.sum((returns - mean) ** 3) / (n * variance ** 1.5)
  kurtosis = np.sum((returns - mean) ** 4) / (n * variance ** 2)
  return mean, variance, skewness, kurtosis

def calculate_skewness_kurtosis(weights, prices):
  # Calculate returns for each asset
  asset_returns = np.apply_along_axis(calculate_returns, 0, prices)

  # Calculate portfolio returns
  portfolio_returns = calculate_portfolio_returns(asset_returns, weights)

  # Calculate moments
  _, _, portfolio_skewness, portfolio_kurtosis = calculate_moments(portfolio_returns)

  return portfolio_skewness, portfolio_kurtosis


In [ ]:
def MO_PSO_Based_Optimisation(asset_prices):
  num_assets = len(asset_prices[0])
  swarm_size = 50

  swarm_returns = np.random.uniform(0, 1, swarm_size) # better upper limit??
  swarm_cardinality = np.random.randint(0.1*num_assets, 0.5*num_assets, swarm_size)

  particle_best_return = np.zeros(swarm_size)
  particle_best_cardinality = np.zeros(swarm_size)
  particle_best_objective = np.zeros(swarm_size)

  for ind in range(swarm_size):
    particle_best_return[ind], particle_best_cardinality[ind] = swarm_returns[ind], swarm_cardinality[ind]

  # bestReturn, bestCardinality, bestPrimary
  min_Risks, initial_weights = lowerLevelSolutionArray(asset_prices, swarm_returns, swarm_cardinality)
  max_primary = -1e12
  best_primary = -1e12
  best_particle_index = 0

  for ind in range(swarm_size):
    particle_best_return[ind], particle_best_cardinality[ind] = swarm_returns[ind], swarm_cardinality[ind]
    skewness, kurtosis = calculate_skewness_kurtosis(initial_weights[ind], asset_prices)
    particle_objective = 0.2*(swarm_returns[ind]/min_Risks[ind]) + (0.4*skewness) + (0.4*(-kurtosis))
    if (particle_objective > max_primary):
      max_primary = particle_objective
      best_particle_index = ind

  best_return = swarm_returns[best_particle_index]
  best_cardinality = swarm_cardinality[best_particle_index]
  best_primary = max_primary

  c1_return = 0.01
  c2_return = 0.01
  c1_cardinality = 0.01
  c2_cardinality = 0.01
  w_return = 0.4
  w_cardinality = 0.4

  improvement = 10
  incLimit = 1e-4
  iterations = 0

  velocity_return = np.random.uniform(-1, 1, swarm_size)
  velocity_cardinality = np.random.uniform(-0.5, 0.5, swarm_size)

  while ((improvement > incLimit or iterations < 10) and iterations < 100):
    for ind in range(swarm_size):
      velocity_return[ind] = w_return*velocity_return[ind] + c1_return*(np.random.uniform(0,1))*(particle_best_return[ind] - swarm_returns[ind]) + c2_return*(np.random.uniform(0,1)*(best_return - swarm_returns[ind]))
      velocity_cardinality[ind] = w_cardinality*velocity_cardinality[ind] + c1_cardinality*(np.random.uniform(0,1))*(particle_best_cardinality[ind] - swarm_cardinality[ind]) + c2_cardinality*(np.random.uniform(0,1)*(best_cardinality - swarm_cardinality[ind]))

      swarm_returns[ind] += velocity_return[ind]
      swarm_cardinality[ind] += velocity_cardinality[ind]

      swarm_cardinality[ind] = int(swarm_cardinality[ind])
      swarm_cardinality[ind] = min(swarm_cardinality[ind], num_assets)
      swarm_cardinality[ind] = max(swarm_cardinality[ind],2)

      swarm_returns[ind] = max(swarm_returns[ind],0)

    min_Risks, weights = lowerLevelSolutionArray(asset_prices, swarm_returns, swarm_cardinality)
    max_primary = -1e12
    best_particle_index = 0

    for ind in range(swarm_size):
      skewness, kurtosis = calculate_skewness_kurtosis(initial_weights[ind], asset_prices)
      particle_objective = 0.2*(swarm_returns[ind]/min_Risks[ind]) + (0.4*skewness) + (0.4*(-kurtosis))

      if (particle_objective > particle_best_objective[ind]):
        particle_best_return[ind] = swarm_returns[ind]
        particle_best_cardinality[ind] = swarm_cardinality[ind]
        particle_best_objective[ind] = particle_objective

      if (particle_objective > max_primary):
        max_primary = particle_objective
        best_particle_index = ind

    improvement = max_primary - best_primary
    if improvement > 0:
      best_primary = max_primary
      best_return = swarm_returns[best_particle_index]
      best_cardinality = swarm_cardinality[best_particle_index]

    iterations += 1

  # print(best_return, best_cardinality, best_primary)

  return best_return, best_cardinality, best_primary

Data Collection

In [ ]:
pricesDF = pd.read_csv('/content/drive/MyDrive/OMF_data_final/uploaded_data_2/Dataset_2_csv.csv')
pricesDF = pricesDF[pricesDF.columns[1:]]

pricesNP = pricesDF.to_numpy()
return_rate = (pricesNP[:-1] - pricesNP[1:])/pricesNP[1:]
exp_return = np.mean(return_rate,axis=0)

sorted_indices = np.argsort(exp_return)
sorted_pricesNP = pricesNP[:, sorted_indices]

In [ ]:
window = 30
num_days = len(pricesDF.index)
num_companies = pricesDF.columns.size
total_rows = num_days - window # Modified(decreased by 1) as we are checking with actual returns

In [ ]:
weights = []

Applying Algorithm

In [ ]:
for i in range(total_rows):
  temp_price_df = pricesDF[i:i+window]
  temp_price_np = temp_price_df.to_numpy()

  temp_return_rate = (temp_price_np[:-1] - temp_price_np[1:])/temp_price_np[1:]
  temp_exp_return = np.mean(temp_return_rate,axis=0)

  temp_variance = np.var(temp_return_rate, axis=0)

  # Sort indices by increasing order of variance
  temp_sorted_indices = np.argsort(temp_variance)

  # Reorder columns of prices array by sorted variance
  temp_sorted_price_np = temp_price_np[:, temp_sorted_indices]

  temp_FinR, temp_FinS, temp_FinP = MO_PSO_Based_Optimisation(temp_sorted_price_np)
  temp_riskInvolved, temp_reqWeights = mean_variance_model_cvxopt(temp_sorted_price_np, temp_FinR, temp_FinS)

  # Reorder `temp_reqWeights` to match the original order
  reordered_weights = np.zeros_like(temp_reqWeights)
  reordered_weights[temp_sorted_indices] = temp_reqWeights

  weights.append(reordered_weights)

<ipython-input-13-94af527ef51c>:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mkw_risk = math.sqrt(wmw.T@temp_cov@wmw)


In [ ]:
weights_df = pd.DataFrame(weights)
weights_df.to_csv("weights_mo_pso_bilevel_convex_combination.csv", index=False, header=False)

In [ ]:
weights_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.0,0.0,0.000000,0.0,0.121140,0.000000,-0.090264,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,0.0,0.0,0.000000,0.0,0.185000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,0.0,0.0,0.000000,0.0,0.159511,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.0,0.151232,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,0.0,0.0,0.078687,0.0,0.078687,0.000000,0.055750,0.078687,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,0.0,0.0,0.000000,0.0,-0.000962,0.056072,0.041717,0.000000,0.0,0.0,...,0.049305,0.0,0.012874,0.0,0.0,0.0,0.0,0.000000,0.032695,0.051157
215,0.0,0.0,0.000000,0.0,-0.018003,0.015148,0.041362,0.000000,0.0,0.0,...,0.000000,0.0,0.022155,0.0,0.0,0.0,0.0,0.000000,0.000000,0.041142
216,0.0,0.0,0.000000,0.0,0.004479,0.197388,0.000669,0.000000,0.0,0.0,...,0.000000,0.0,0.000083,0.0,0.0,0.0,0.0,0.000000,0.000000,0.011276
217,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.208743


In [24]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')